In [4]:
import mysql.connector
def get_db_connection():
    return mysql.connector.connect(
        host="localhost",
        user="root",
        password="1882x09",
        database="parking"
    )

In [21]:
from flask import Flask , request, jsonify
import os
import base64
from io import BytesIO
from PIL import Image
from ultralytics import YOLO
import cv2 
from datetime import datetime, timezone
app=Flask(__name__)


@app.route("/ver_entradas", methods=["GET"])
def ver_entradas():
    try:
        conn = get_db_connection()
        cursor = conn.cursor(dictionary=True)
        cursor.execute("SELECT * FROM parking_tienda")
        resultados = cursor.fetchall()
        cursor.close()
        conn.close()
        return jsonify(resultados)
    except Exception as e:
        return jsonify({"error": str(e)}), 500

@app.route("/parking_entrada", methods=["POST"])
def entradaParking():
    try:

        conn = get_db_connection()
        cursor = conn.cursor(dictionary=True)
        data = request.json
        base64_string = data.get("imagen")
        if not base64_string :
            return jsonify({"error": "Faltan datos (imagen)"}), 400

        # Decodificar Base64 a imagen
        image_bytes = base64.b64decode(base64_string)
        image = Image.open(BytesIO(image_bytes))

        # Guardar la imagen como PNG
        actual_path = os.path.abspath(os.getcwd())
        image_path = os.path.join(actual_path, "imagen_procesada.png")
        image.save(image_path, "PNG")
        
        # Procesar la imagen con YOLO
        model_Path='/home/david/Escritorio/programacion/parking/models/get-plate-model.pt'
        model=YOLO(model_Path)
        source_image=actual_path+'/testing/old-car.jpeg'
        print(source_image)
        """
        images=[
            'data/val/images/ba2b2dbd-captura_20250318-161749.png',
            'data/val/images/be627c47-captura_20250318-161845.png'
        ]
        """
        results = model.predict(
            source=source_image,
            conf=0.25,    # Umbral de confianza
            save=False,   # No que dibuje YOLO; lo haremos manualmente con OpenCV
            verbose=False
        )
        orig_img=cv2.imread(source_image)
        orig_img=cv2.cvtColor(orig_img, cv2.COLOR_BGR2RGB)

        predictions = results[0]
        detecto_placa = False  # Variable para verificar si hay detecciones
        for box in predictions.boxes:
            detecto_placa = True
            x1, y1, x2, y2 = box.xyxy[0]

            # Convertir a int para dibujar con OpenCV
            x1, y1, x2, y2 = int(x1), int(y1), int(x2), int(y2)

            print(x1, y1, x2, y2)

            # ID de la clase detectada
            cls_id = int(box.cls[0])

            # Confianza de la detección
            conf = float(box.conf[0])

            # Nombre de la clase (basado en model.names)
            class_name = model.names[cls_id] if model.names and cls_id < len(model.names) else f"cls_{cls_id}"

            print(f"Clase: {class_name}, confianza: {conf:.2f}")
            # Dibujo del rectángulo
            color = (0, 255, 0)  # Verde
            thickness = 2
            cv2.rectangle(orig_img, (x1, y1), (x2, y2), color, thickness)

            # Texto (etiqueta + confianza)
            label = f"{class_name} {conf:.2f}"
            # Para dibujar el fondo del texto (opcional, para que sea legible)
            (tw, th), baseline = cv2.getTextSize(label, cv2.FONT_HERSHEY_SIMPLEX, 0.5, 1)
            # Caja para el texto encima del bounding box
            cv2.rectangle(orig_img, (x1, y1 - th - baseline), (x1 + tw, y1), color, -1)
            cv2.putText(orig_img, label, (x1, y1 - baseline),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 0), 1)

            # Si se detecta una caja, se recorta la imagen
            img_NZ_bgr = cv2.imread(source_image, cv2.IMREAD_COLOR)
            img_NZ_rgb = img_NZ_bgr[:, :, ::-1]  # a RGB

            plate = img_NZ_rgb[y1:y2, x1:x2].copy()
            cv2.imwrite('/home/david/Escritorio/programacion/parking/testing/new_plate224.png', plate)
            
            cursor.execute(
                "SELECT * from parking_tienda where	matricula = %s",
                ("1234ABC",))
            buscar_matricula = cursor.fetchone()
            if buscar_matricula:
                return jsonify({"error": "La matrícula ya está registrada"}), 400
            else:
                get_dateTime=  datetime.now(timezone.utc)
                fecha = str(get_dateTime.date())
                hora = get_dateTime.strftime("%H:%M:%S")
        
                if detecto_placa:
                    cursor.execute(
                    "INSERT INTO parking_tienda (matricula, hora_entrada,fecha_entrada) VALUES (%s, %s, %s)",
                    ("9874JDF", hora, fecha)
                )
                conn.commit()

                cursor.close()
                conn.close()

        return jsonify({    
            "message": "Imagen procesada y guardada en la base de datos",
        })

    except Exception as e:
        return jsonify({"error": str(e)}), 500
    
@app.route("/parking_salida", methods=["POST"])
def salidaParking():
    try:
        conn = get_db_connection()
        cursor = conn.cursor(dictionary=True)
        data = request.json
        base64_string = data.get("imagen")
        hora = data.get("hora")

        if not base64_string :
            return jsonify({"error": "Faltan datos (imagen)"}), 400

        # Decodificar Base64 a imagen
        image_bytes = base64.b64decode(base64_string)
        image = Image.open(BytesIO(image_bytes))

        # Guardar la imagen como PNG
        actual_path = os.path.abspath(os.getcwd())
        image_path = os.path.join(actual_path, "imagen_procesada.png")
        image.save(image_path, "PNG")
        
        # Procesar la imagen con YOLO
        model_Path='/home/david/Escritorio/programacion/parking/models/get-plate-model.pt'
        model=YOLO(model_Path)
        source_image=actual_path+'/testing/old-car.jpeg'
        print(source_image)
        """
        images=[
            'data/val/images/ba2b2dbd-captura_20250318-161749.png',
            'data/val/images/be627c47-captura_20250318-161845.png'
        ]
        """
        results = model.predict(
            source=source_image,
            conf=0.25,    # Umbral de confianza
            save=False,   # No que dibuje YOLO; lo haremos manualmente con OpenCV
            verbose=False
        )
        orig_img=cv2.imread(source_image)
        orig_img=cv2.cvtColor(orig_img, cv2.COLOR_BGR2RGB)

        predictions = results[0]
        detecto_placa = False  # Variable para verificar si hay detecciones
        for box in predictions.boxes:
            detecto_placa = True
            x1, y1, x2, y2 = box.xyxy[0]

            # Convertir a int para dibujar con OpenCV
            x1, y1, x2, y2 = int(x1), int(y1), int(x2), int(y2)

            print(x1, y1, x2, y2)

            # ID de la clase detectada
            cls_id = int(box.cls[0])

            # Confianza de la detección
            conf = float(box.conf[0])

            # Nombre de la clase (basado en model.names)
            class_name = model.names[cls_id] if model.names and cls_id < len(model.names) else f"cls_{cls_id}"

            print(f"Clase: {class_name}, confianza: {conf:.2f}")
            # Dibujo del rectángulo
            color = (0, 255, 0)  # Verde
            thickness = 2
            cv2.rectangle(orig_img, (x1, y1), (x2, y2), color, thickness)

            # Texto (etiqueta + confianza)
            label = f"{class_name} {conf:.2f}"
            # Para dibujar el fondo del texto (opcional, para que sea legible)
            (tw, th), baseline = cv2.getTextSize(label, cv2.FONT_HERSHEY_SIMPLEX, 0.5, 1)
            # Caja para el texto encima del bounding box
            cv2.rectangle(orig_img, (x1, y1 - th - baseline), (x1 + tw, y1), color, -1)
            cv2.putText(orig_img, label, (x1, y1 - baseline),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 0), 1)

            # Si se detecta una caja, se recorta la imagen
            img_NZ_bgr = cv2.imread(source_image, cv2.IMREAD_COLOR)
            img_NZ_rgb = img_NZ_bgr[:, :, ::-1]  # a RGB

            plate = img_NZ_rgb[y1:y2, x1:x2].copy()
            cv2.imwrite('/home/david/Escritorio/programacion/parking/testing/new_plate224.png', plate)

            if detecto_placa:
               cursor.execute(
                "SELECT * from parking_tienda where	matricula = %s",
                ("1234ABC",))
               buscar_matricula = cursor.fetchone()
          
            if buscar_matricula:
                get_dateTime=  datetime.now(timezone.utc)
                hora = get_dateTime.strftime("%H:%M:%S")
                cursor.execute(
                "UPDATE parking_tienda SET hora_salida = %s WHERE matricula = %s", 
                (hora, "1234ABC"))
                conn.commit()
                conn.commit()
                cursor.close()
                conn.close()
                return   jsonify({     
                    "message": "Imagen procesada y guardada en la base de datos",

                })
                
            else:
                conn.commit()
                cursor.close()
                conn.close()
                return jsonify({"error": "No se encontró la matrícula"}), 404
            
                
    except Exception as e:
        return jsonify({"error": str(e)}), 500    

if __name__=="__main__":
    app.run()   # localhost at 5000

 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit


/home/david/Escritorio/programacion/parking/testing/old-car.jpeg


127.0.0.1 - - [06/Apr/2025 19:09:16] "POST /parking_entrada HTTP/1.1" 400 -


543 331 669 389
Clase: 15, confianza: 0.95
